In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#     /content/drive/MyDrive/datasets/BUILDING DEFECTS PROJECT/Historical Building Cracks.zip

###DETECTING BUILDING DEFECTS USING VGG16###

In [ ]:
!unzip "/content/drive/MyDrive/datasets/BUILDING DEFECTS PROJECT/Historical Building Cracks.zip"

Archive:  /content/drive/MyDrive/datasets/BUILDING DEFECTS PROJECT/Historical Building Cracks.zip
   creating: Historical Building Cracks/
   creating: Historical Building Cracks/crack/
  inflating: Historical Building Cracks/crack/1002.jpg  
  inflating: Historical Building Cracks/crack/1012.jpg  
  inflating: Historical Building Cracks/crack/1014.jpg  
  inflating: Historical Building Cracks/crack/10295.jpg  
  inflating: Historical Building Cracks/crack/10413.jpg  
  inflating: Historical Building Cracks/crack/10438.jpg  
  inflating: Historical Building Cracks/crack/1051.jpg  
  inflating: Historical Building Cracks/crack/10552.jpg  
  inflating: Historical Building Cracks/crack/10616.jpg  
  inflating: Historical Building Cracks/crack/10636.jpg  
  inflating: Historical Building Cracks/crack/10656.jpg  
  inflating: Historical Building Cracks/crack/1068.jpg  
  inflating: Historical Building Cracks/crack/10766.jpg  
  inflating: Historical Building Cracks/crack/10786.jpg  
  infla

###A python library used to split Train and Test also Validation Data (Source: Google)###

In [ ]:
pip install split-folders[full]

In [ ]:
import splitfolders

input_folder = "/content/Historical Building Cracks" #Enter Input Folder
output = "/content/DATASET" #Enter Output Folder

splitfolders.ratio(input_folder, output=output, seed=42, ratio=(0.8,0.1,0.1))

Copying files: 3896 files [00:00, 9147.80 files/s]


###DATA AUGMENTATION###

In [ ]:
# Data Augmentation

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_gen = ImageDataGenerator(rescale=(1./255),horizontal_flip=True,shear_range=0.2)
test_gen = ImageDataGenerator(rescale=(1./255))

In [ ]:
train = train_gen.flow_from_directory('/content/DATASET/train',
                                      target_size=(120, 120),
                                      class_mode='categorical',
                                      batch_size=8)
test = test_gen.flow_from_directory('/content/DATASET/test',
                                    target_size=(120, 120),
                                      class_mode='categorical',
                                      batch_size=8)

Found 3116 images belonging to 2 classes.
Found 392 images belonging to 2 classes.


In [ ]:
train.class_indices

{'crack': 0, 'non-crack': 1}

###MODEL BUILDING###

In [ ]:
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Convolution2D(20,(3,3),activation='relu',input_shape=(120, 120, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(45,activation='relu'))
model.add(Dense(2,activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(train,batch_size=34,validation_data=test,epochs=10)

Epoch 1/10
390/390 [==============================] - 36s 53ms/step - loss: 0.6796 - accuracy: 0.7953 - val_loss: 0.4901 - val_accuracy: 0.8036
Epoch 2/10
390/390 [==============================] - 21s 55ms/step - loss: 0.5289 - accuracy: 0.8058 - val_loss: 0.5287 - val_accuracy: 0.8036
Epoch 3/10
390/390 [==============================] - 15s 39ms/step - loss: 0.5108 - accuracy: 0.8058 - val_loss: 0.5031 - val_accuracy: 0.8036
Epoch 4/10
390/390 [==============================] - 15s 39ms/step - loss: 0.4967 - accuracy: 0.8058 - val_loss: 0.4969 - val_accuracy: 0.8036
Epoch 5/10
390/390 [==============================] - 17s 43ms/step - loss: 0.4932 - accuracy: 0.8058 - val_loss: 0.4956 - val_accuracy: 0.8036
Epoch 6/10
390/390 [==============================] - 16s 41ms/step - loss: 0.4924 - accuracy: 0.8058 - val_loss: 0.4954 - val_accuracy: 0.8036
Epoch 7/10
390/390 [==============================] - 16s 40ms/step - loss: 0.4923 - accuracy: 0.8058 - val_loss: 0.4954 - val_accuracy:

In [ ]:
model.save('BuildingWeights.h5')

###Testing the Model###

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

In [ ]:
# Testing 1
img1 = image.load_img('/content/DATASET/val/non-crack/11472 - Copy.jpg',target_size=(120,120))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred = np.argmax(model.predict(img1))
print(pred)
output = ['Cracked','Non-cracked']
print(output[pred])

if output[pred]=='Cracked':
  print("Building Has a Defect!!! Better repair it!")
else:
  print("Building has no Defect!!!")

1/1 [==============================] - 0s 160ms/step
1
Non-cracked
Building has no Defect!!!


In [ ]:
# Testing 1
img2 = image.load_img('/content/DATASET/test/crack/13777.jpg',target_size=(120,120))
img2 = image.img_to_array(img2)
img2 = np.expand_dims(img2,axis=0)
pred = np.argmax(model.predict(img2))
print(pred)
output = ['Cracked','Non-cracked']
print(output[pred])

if output[pred]=='Cracked':
  print("Building Has a Defect!!! Better repair it!")
else:
  print("Building has no Defect!!!")

1/1 [==============================] - 0s 29ms/step
1
Non-cracked
Building has no Defect!!!


###Model Tuning###

In [ ]:
from tensorflow.keras.layers import BatchNormalization, Dropout

In [ ]:
# Initializing the seq model
model = Sequential()
# Adding conv layer with input
model.add(Convolution2D(12,(3,3),activation='relu',input_shape=(120, 120, 3)))
# Normalizing the conv layer output
model.add(BatchNormalization())
# Selecting the max values
model.add(MaxPooling2D(pool_size=(2,2)))
# Dropping the unwanted 20% of data
model.add(Dropout(0.2))
model.add(Convolution2D(24,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Convolution2D(36,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
# Hiddern layers
model.add(Dense(62,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
# Output layer
model.add(Dense(2,activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 118, 118, 12)      336       
                                                                 
 batch_normalization (BatchN  (None, 118, 118, 12)     48        
 ormalization)                                                   
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 59, 59, 12)       0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 59, 59, 12)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 57, 57, 24)        2616      
                                                                 
 batch_normalization_1 (Batc  (None, 57, 57, 24)      

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# Early stopping
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='accuracy',patience=4)

In [ ]:
model.fit(train,batch_size=34,validation_data=test,epochs=50,callbacks=early_stop)

Epoch 1/50
390/390 [==============================] - 22s 42ms/step - loss: 0.4179 - accuracy: 0.8280 - val_loss: 1.0990 - val_accuracy: 0.8087
Epoch 2/50
390/390 [==============================] - 16s 41ms/step - loss: 0.3157 - accuracy: 0.8790 - val_loss: 0.4700 - val_accuracy: 0.8087
Epoch 3/50
390/390 [==============================] - 16s 41ms/step - loss: 0.2439 - accuracy: 0.9047 - val_loss: 0.4255 - val_accuracy: 0.8648
Epoch 4/50
390/390 [==============================] - 16s 41ms/step - loss: 0.2258 - accuracy: 0.9223 - val_loss: 0.2651 - val_accuracy: 0.9107
Epoch 5/50
390/390 [==============================] - 16s 41ms/step - loss: 0.1882 - accuracy: 0.9361 - val_loss: 0.3114 - val_accuracy: 0.9337
Epoch 6/50
390/390 [==============================] - 16s 41ms/step - loss: 0.1617 - accuracy: 0.9451 - val_loss: 0.3316 - val_accuracy: 0.8878
Epoch 7/50
390/390 [==============================] - 18s 46ms/step - loss: 0.1781 - accuracy: 0.9326 - val_loss: 0.8245 - val_accuracy:

###Testing the tuned Model###

In [ ]:
# Testing 1
img1 = image.load_img('/content/DATASET/val/non-crack/11472 - Copy.jpg',target_size=(120,120))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred = np.argmax(model.predict(img1))
print(pred)
output = ['Cracked','Non-cracked']
print(output[pred])

if output[pred]=='Cracked':
  print("Building Has a Defect!!! Better repair it!")
else:
  print("Building has no Defect!!!")

1/1 [==============================] - 0s 210ms/step
1
Non-cracked
Building has no Defect!!!


In [ ]:
# Testing 2
img2 = image.load_img('/content/DATASET/test/crack/1969.jpg',target_size=(120,120))
img2 = image.img_to_array(img2)
img2 = np.expand_dims(img2,axis=0)
pred = np.argmax(model.predict(img2))
print(pred)
output = ['Cracked','Non-cracked']
print(output[pred])

if output[pred]=='Cracked':
  print("Building Has a Defect!!! Better repair it!")
else:
  print("Building has no Defect!!!")

1/1 [==============================] - 0s 18ms/step
0
Cracked
Building Has a Defect!!! Better repair it!


In [ ]:
img3 = image.load_img('/content/DATASET/test/crack/1792.jpg',target_size=(120,120))
img3 = image.img_to_array(img3)
img3 = np.expand_dims(img3,axis=0)
pred = np.argmax(model.predict(img3))
print(pred)
output = ['Cracked','Non-cracked']
print(output[pred])

if output[pred]=='Cracked':
  print("Building Has a Defect!!! Better repair it!")
else:
  print("Building has no Defect!!!")

1/1 [==============================] - 0s 35ms/step
0
Cracked
Building Has a Defect!!! Better repair it!


In [ ]:
img4 = image.load_img('/content/DATASET/test/crack/9325.jpg',target_size=(120,120))
img4 = image.img_to_array(img4)
img4 = np.expand_dims(img4,axis=0)
pred = np.argmax(model.predict(img4))
print(pred)
output = ['Cracked','Non-cracked']
print(output[pred])

if output[pred]=='Cracked':
  print("Building Has a Defect!!! Better repair it!")
else:
  print("Building has no Defect!!!")

1/1 [==============================] - 0s 33ms/step
1
Non-cracked
Building has no Defect!!!


In [ ]:
img5 = image.load_img('/content/DATASET/test/crack/1457.jpg',target_size=(120,120))
img5 = image.img_to_array(img5)
img5 = np.expand_dims(img5,axis=0)
pred = np.argmax(model.predict(img5))
print(pred)
output = ['Cracked','Non-cracked']
print(output[pred])

if output[pred]=='Cracked':
  print("Building Has a Defect!!! Better repair it!")
else:
  print("Building has no Defect!!!")

1/1 [==============================] - 0s 30ms/step
1
Non-cracked
Building has no Defect!!!


###**TRANSFER** **LEARNING VGG16**###

In [ ]:
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import numpy as np

In [ ]:
train_path = "/content/DATASET/train"
test_path = "/content/DATASET/test"

In [ ]:
train_gen = ImageDataGenerator(rescale=1./255,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True)

test_gen = ImageDataGenerator(rescale=1./255)

In [ ]:

train = train_gen.flow_from_directory(train_path,
                                      target_size=(224,224),
                                      batch_size=22,
                                      class_mode='categorical')

test = test_gen.flow_from_directory(test_path,
                                      target_size=(224,224),
                                      batch_size=22,
                                      class_mode='categorical')

Found 3116 images belonging to 2 classes.
Found 392 images belonging to 2 classes.


### Model Building###

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

In [ ]:
vgg = VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))

58889256/58889256 [==============================] - 1s 0us/step


In [ ]:
for layer in vgg.layers:
  print(layer)

In [ ]:
# Train model with existing weights

for layer in vgg.layers:
  layer.trainable=False

In [ ]:
x = Flatten()(vgg.output)

In [ ]:
# output layer

prediction = Dense(2,activation='softmax')(x)

In [ ]:
# Create Vgg16 model

model_vgg16 = Model(inputs=vgg.input,outputs=prediction)

In [ ]:
model_vgg16.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
model_vgg16.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model_vgg16.fit_generator(train,validation_data=test,epochs=10,steps_per_epoch=len(train),validation_steps=len(test))

<ipython-input-42-1aa761bf7422>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_vgg16.fit_generator(train,validation_data=test,epochs=10,steps_per_epoch=len(train),validation_steps=len(test))


Epoch 1/10
142/142 [==============================] - 54s 345ms/step - loss: 0.2125 - accuracy: 0.9268 - val_loss: 0.1580 - val_accuracy: 0.9541
Epoch 2/10
142/142 [==============================] - 43s 300ms/step - loss: 0.1155 - accuracy: 0.9605 - val_loss: 0.1198 - val_accuracy: 0.9668
Epoch 3/10
142/142 [==============================] - 43s 304ms/step - loss: 0.0984 - accuracy: 0.9685 - val_loss: 0.0716 - val_accuracy: 0.9796
Epoch 4/10
142/142 [==============================] - 43s 304ms/step - loss: 0.0785 - accuracy: 0.9718 - val_loss: 0.0676 - val_accuracy: 0.9796
Epoch 5/10
142/142 [==============================] - 43s 304ms/step - loss: 0.0740 - accuracy: 0.9743 - val_loss: 0.0720 - val_accuracy: 0.9745
Epoch 6/10
142/142 [==============================] - 44s 307ms/step - loss: 0.1161 - accuracy: 0.9657 - val_loss: 0.1818 - val_accuracy: 0.9490
Epoch 7/10
142/142 [==============================] - 45s 314ms/step - loss: 0.0877 - accuracy: 0.9705 - val_loss: 0.0732 - val_ac

In [ ]:
model_vgg16.save('BuildingWeights_vgg16.h5')

### Testing the VGG16 Model###

In [ ]:
# Testing 1
img1 = image.load_img('/content/DATASET/test/crack/1486.jpg',target_size=(224,224))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred = np.argmax(model_vgg16.predict(img1))
print(pred)
output = ['Cracked','Non-cracked']
print(output[pred])

if output[pred]=='Cracked':
  print("Building Has a Defect!!! Better repair it!")
else:
  print("Building has no Defect!!!")

1/1 [==============================] - 1s 714ms/step
0
Cracked
Building Has a Defect!!! Better repair it!


In [ ]:
# Testing 2
img2 = image.load_img('/content/DATASET/test/crack/15092.jpg',target_size=(224,224))
img2 = image.img_to_array(img2)
img2 = np.expand_dims(img2,axis=0)
pred = np.argmax(model_vgg16.predict(img2))
print(pred)
output = ['Cracked','Non-cracked']
print(output[pred])

if output[pred]=='Cracked':
  print("Building Has a Defect!!! Better repair it!")
else:
  print("Building has no Defect!!!")

1/1 [==============================] - 0s 21ms/step
0
Cracked
Building Has a Defect!!! Better repair it!


In [ ]:
# Testing 3
img3 = image.load_img('/content/DATASET/test/non-crack/11041.jpg',target_size=(224,224))
img3 = image.img_to_array(img3)
img3 = np.expand_dims(img3,axis=0)
pred = np.argmax(model_vgg16.predict(img3))
print(pred)
output = ['Cracked','Non-cracked']
print(output[pred])

if output[pred]=='Cracked':
  print("Building Has a Defect!!! Better repair it!")
else:
  print("Building has no Defect!!!")

1/1 [==============================] - 0s 20ms/step
1
Non-cracked
Building has no Defect!!!


In [ ]:
# Testing4
img4 = image.load_img('/content/DATASET/test/crack/2010.jpg',target_size=(224,224))
img4 = image.img_to_array(img4)
img4 = np.expand_dims(img4,axis=0)
pred = np.argmax(model_vgg16.predict(img4))
print(pred)
output = ['Cracked','Non-cracked']
print(output[pred])

if output[pred]=='Cracked':
  print("Building Has a Defect!!! Better repair it!")
else:
  print("Building has no Defect!!!")

1/1 [==============================] - 0s 24ms/step
0
Cracked
Building Has a Defect!!! Better repair it!


In [ ]:
if output[pred]=='Cracked':
  print("Building Has a Defect!!! Better repair it!")
else:
  print("Building has no Defect!!!")

Building Has a Defect!!! Better repair it!
